*INSPIRED BY https://github.com/BogdanJeN/Geo/blob/main/Lab_06/lab_06.ipynb REPOSITORY* 😇

imports (osgeo - gdal)

In [72]:

from osgeo import gdal
from osgeo import ogr
from osgeo import osr
import matplotlib
import rasterio
import geopandas as gpd
from shapely.geometry import Point, mapping
import json # to save list into .geojson file

read data

In [73]:
soil_data = {
    "clay_data": rasterio.open("./soil_data/clay.tif"),
    "density_data": rasterio.open("./soil_data/density.tif"),
    "sand_data": rasterio.open("./soil_data/sand.tif"),
    "moisture_data": rasterio.open("./soil_data/soil_moisture.tif")
}

centroid_data = gpd.read_file("./centroids/field_centroids.geojson")

update our objects from geojson file with new data (reasters, coordinates)

In [74]:
# c = soil_data["clay_data"]
# row, col = c.index(30.250268297428843, 49.989318439198321)
# print(c.read(1)[row][col])

new_geojson_lst = [] # create empty list to contain our new future data

for i, row in centroid_data.iterrows():
    row_id = row["id"]
    row_name = row["Name"]
    geometry = row["geometry"].centroid
    lat, lon = row['geometry'].y, row['geometry'].x # get coordinates of the field
    
    row, col = soil_data["clay_data"].index(lon, lat) # get pixel coordinates of the clay data
    
    # read single pixels from rasters using a window argument
    sand_coordinates = float(soil_data["sand_data"].read(1, window=((row, row+1), (col, col+1))))
    clay_coordinates = float(soil_data["clay_data"].read(1, window=((row, row+1), (col, col+1))))
    density_coordinates = int(soil_data["density_data"].read(1, window=((row, row+1), (col, col+1))))
    
    row, col = soil_data["sand_data"].index(lon, lat) # get pixel coordinates of the sand data
    
    moisture_coordinates = int(soil_data["moisture_data"].read(1, window=((row, row+1), (col, col+1))))  # read single pixel from moisture raster using a window argument
    
    new_geojson_obj = { # update object with new data
        "type": "Feature",
        "properties": {
            "id": row_id,
            "name": row_name,
            "soil": { # new
                "clay": clay_coordinates,
                "sand": sand_coordinates,
                "density": density_coordinates,
            },
            "soil_moisture": moisture_coordinates, # new
            "coordinates": { # new
                "lat": lat, 
                "lon": lon
            }
        },
        "geometry": {
            "type": "Point",
            "coordinates": mapping(geometry)
        }
    }
    new_geojson_lst.append(new_geojson_obj) # add updated obj to the list

Create future collection to save our list in GeoJSON format in valid way

In [75]:
# this object will help the programm to understand in what extension we want to save our list
feature_collection = {
    "type": "FeatureCollection", # Identifies the object as a GeoJSON Featue Collection and provides context for the features that are included in the file
    "features": new_geojson_lst # our list
}

Save our data into the new file

In [77]:
# save using 'json' import
with open("./centroids/updated_field_centroids.geojson", "w") as output_file:
    json.dump(feature_collection, output_file, indent=2) # need this 'indent=2' to beutify our .geojson file